## Market Basket Analysis

### Introduction: In this Market Basket Analysis we look at the products purchased by several customers from a retail location and then use the data to find associations among the products purchased. For example the analysis may answer questions such as, given a customer has purchased "onions" what is their likelihood of purchasing "whole milk".

### Step 1: Import Data File in CSV Format

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


### Step 2: Install PANDAS package MLXTEND and import its subpackages needed for Market Basket Analysis

In [2]:
!pip install pandas mlxtend

     |████████████████████████████████| 1.3 MB 12.3 MB/s eta 0:00:01


In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Step 3: DATA WRANGLING that includes: (a) Dropping Duplicate Rows; (b) Renaming Columns; (c) Dropping Rows with Missing Data; (d) Stripping purchase names of leading and trailing white spaces; and (e) Dropping DATE column which is unneeded in our Model.

In [32]:
df = df_data_1

In [33]:
df = df.drop_duplicates()

In [34]:
df.shape

(38006, 3)

In [35]:
df.columns = ["ID","DATE","PURCHASE"]

In [36]:
df.reset_index(drop=True, inplace=True)

In [37]:
df = df[df["ID"] != None]

In [38]:
df = df[df["PURCHASE"] != None]

In [39]:
df["PURCHASE"].value_counts()

whole milk               2363
other vegetables         1827
rolls/buns               1646
soda                     1453
yogurt                   1285
                         ... 
make up remover             5
bags                        4
baby cosmetics              3
kitchen utensil             1
preservation products       1
Name: PURCHASE, Length: 167, dtype: int64

In [40]:
df = pd.DataFrame(df)

In [41]:
df["PURCHASE"].str.strip()

0               tropical fruit
1                   whole milk
2                    pip fruit
3             other vegetables
4                   whole milk
                 ...          
38001            sliced cheese
38002                    candy
38003                 cake bar
38004    fruit/vegetable juice
38005                 cat food
Name: PURCHASE, Length: 38006, dtype: object

In [42]:
df = pd.DataFrame(df)

In [43]:
df.head()

,ID,DATE,PURCHASE
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [44]:
df.drop(['DATE'],axis = 1, inplace = True)

In [45]:
df.head()

,ID,PURCHASE
0,1808,tropical fruit
1,2552,whole milk
2,2300,pip fruit
3,1187,other vegetables
4,3037,whole milk


In [46]:
df.shape

(38006, 2)

In [47]:
df.dtypes

ID           int64
PURCHASE    object
dtype: object

### Step 4: Grouping Customer PURCHASES Data on ID Numbers Key and sending result to a LIST. 

In [48]:
df = df.groupby('ID')['PURCHASE'].apply(list)

In [49]:
type(df)

pandas.core.series.Series

In [50]:
df_ = list()
for _ in df:
  #using list comprehension 
  _ = [x for x in _ if x is not None]
  df_.append(_)
df = df_
#df

### Step 5: Using ONE-HOT ENCODING encode customer PURCHASES

In [51]:
te = TransactionEncoder()
te_ary = te.fit(df).transform(df)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3894,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
3895,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3896,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,False


In [52]:
df.shape

(3898, 167)

### Step 6: Creating Frequency Table of Items Purchased Using apriori function. Lowest frequency considered = 0.01.

In [53]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.015393,(Instant food products)
1,0.078502,(UHT-milk)
2,0.031042,(baking powder)
3,0.119548,(beef)
4,0.079785,(berries)
...,...,...
3011,0.011031,"(whole milk, soda, yogurt, whipped/sour cream)"
3012,0.010518,"(whole milk, bottled water, other vegetables, ..."
3013,0.013597,"(whole milk, other vegetables, sausage, yogurt..."
3014,0.010005,"(whole milk, other vegetables, yogurt, shoppin..."


In [26]:
frequent_itemsets['support'].min()

0.010005130836326322

### Step 7: Associating Purchase Items in order of antecedents (items purchased before) and consequents (items purchased after). This uses the association_rules function from mxltrend.

In [27]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(UHT-milk),(other vegetables),0.078502,0.376603,0.038994,0.496732,1.318979,0.009430,1.238697
1,(UHT-milk),(whole milk),0.078502,0.458184,0.040534,0.516340,1.126928,0.004565,1.120242
2,(baking powder),(other vegetables),0.031042,0.376603,0.015136,0.487603,1.294740,0.003446,1.216629
3,(baking powder),(whole milk),0.031042,0.458184,0.015136,0.487603,1.064209,0.000913,1.057416
4,(beef),(other vegetables),0.119548,0.376603,0.050795,0.424893,1.128223,0.005773,1.083966
...,...,...,...,...,...,...,...,...,...
2238,"(rolls/buns, shopping bags, whole milk, yogurt)",(other vegetables),0.016932,0.376603,0.010005,0.590909,1.569049,0.003629,1.523858
2239,"(yogurt, rolls/buns, soda, other vegetables)",(whole milk),0.020010,0.458184,0.013597,0.679487,1.483002,0.004428,1.690467
2240,"(yogurt, whole milk, soda, other vegetables)",(rolls/buns),0.027963,0.349666,0.013597,0.486239,1.390578,0.003819,1.265828
2241,"(whole milk, rolls/buns, soda, other vegetables)",(yogurt),0.031811,0.282966,0.013597,0.427419,1.510499,0.004595,1.252285


In [54]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(beef),(UHT-milk),0.119548,0.078502,0.010518,0.087983,1.120775,0.001133,1.010396
1,(UHT-milk),(beef),0.078502,0.119548,0.010518,0.133987,1.120775,0.001133,1.016672
2,(bottled beer),(UHT-milk),0.158799,0.078502,0.014879,0.093700,1.193597,0.002413,1.016769
3,(UHT-milk),(bottled beer),0.078502,0.158799,0.014879,0.189542,1.193597,0.002413,1.037933
4,(bottled water),(UHT-milk),0.213699,0.078502,0.021293,0.099640,1.269268,0.004517,1.023477
...,...,...,...,...,...,...,...,...,...
15475,(whole milk),"(soda, rolls/buns, yogurt, other vegetables)",0.458184,0.020010,0.013597,0.029675,1.483002,0.004428,1.009961
15476,(other vegetables),"(whole milk, rolls/buns, yogurt, soda)",0.376603,0.024628,0.013597,0.036104,1.465954,0.004322,1.011905
15477,(soda),"(whole milk, rolls/buns, yogurt, other vegetab...",0.313494,0.034377,0.013597,0.043372,1.261658,0.002820,1.009403
15478,(yogurt),"(whole milk, rolls/buns, soda, other vegetables)",0.282966,0.031811,0.013597,0.048051,1.510499,0.004595,1.017059


In [55]:
rules["lift"].max()

2.4286889871155837

In [57]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(beef),(UHT-milk),0.119548,0.078502,0.010518,0.087983,1.120775,0.001133,1.010396,1
1,(UHT-milk),(beef),0.078502,0.119548,0.010518,0.133987,1.120775,0.001133,1.016672,1
2,(bottled beer),(UHT-milk),0.158799,0.078502,0.014879,0.093700,1.193597,0.002413,1.016769,1
3,(UHT-milk),(bottled beer),0.078502,0.158799,0.014879,0.189542,1.193597,0.002413,1.037933,1
4,(bottled water),(UHT-milk),0.213699,0.078502,0.021293,0.099640,1.269268,0.004517,1.023477,1
...,...,...,...,...,...,...,...,...,...,...
15475,(whole milk),"(soda, rolls/buns, yogurt, other vegetables)",0.458184,0.020010,0.013597,0.029675,1.483002,0.004428,1.009961,1
15476,(other vegetables),"(whole milk, rolls/buns, yogurt, soda)",0.376603,0.024628,0.013597,0.036104,1.465954,0.004322,1.011905,1
15477,(soda),"(whole milk, rolls/buns, yogurt, other vegetab...",0.313494,0.034377,0.013597,0.043372,1.261658,0.002820,1.009403,1
15478,(yogurt),"(whole milk, rolls/buns, soda, other vegetables)",0.282966,0.031811,0.013597,0.048051,1.510499,0.004595,1.017059,1


### Step 8: Finally we create a Rule as per our choice which will influence antcedents and consequents of items purchased. This can be used to inform us about what items are purchased which lead to or follow from the purchase of other items.

In [58]:
rules[ (rules['antecedent_len'] >= 0) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 0.9) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
57,(UHT-milk),(whole milk),0.078502,0.458184,0.040534,0.516340,1.126928,0.004565,1.120242,1
136,(beef),(whole milk),0.119548,0.458184,0.064135,0.536481,1.170886,0.009360,1.168919,1
331,(bottled beer),(whole milk),0.158799,0.458184,0.085428,0.537964,1.174124,0.012669,1.172672,1
439,(bottled water),(whole milk),0.213699,0.458184,0.112365,0.525810,1.147597,0.014452,1.142615,1
517,(brown bread),(whole milk),0.135967,0.458184,0.069779,0.513208,1.120091,0.007481,1.113034,1
...,...,...,...,...,...,...,...,...,...,...
15420,"(whole milk, shopping bags, yogurt, other vege...",(rolls/buns),0.017958,0.349666,0.010005,0.557143,1.593355,0.003726,1.468495,4
15423,"(whole milk, shopping bags, rolls/buns, yogurt)",(other vegetables),0.016932,0.376603,0.010005,0.590909,1.569049,0.003629,1.523858,4
15424,"(rolls/buns, shopping bags, yogurt, other vege...",(whole milk),0.014110,0.458184,0.010005,0.709091,1.547613,0.003540,1.862494,4
15453,"(whole milk, rolls/buns, yogurt, soda)",(other vegetables),0.024628,0.376603,0.013597,0.552083,1.465954,0.004322,1.391769,4


### For example below are the items purchased by customers who also purchased whole milk and bottled water in the past

In [59]:
rules[rules['antecedents'] == {'bottled water','whole milk'}]

xx = rules[rules['antecedents'] == {'bottled water', 'whole milk'}]

#xx = xx[xx['confidence'] == xx['confidence'].max()]

xx = xx[xx['confidence'] >= 0.3]

xx


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3992,"(whole milk, bottled water)",(other vegetables),0.112365,0.376603,0.056183,0.500000,1.327657,0.013865,1.246793,2
4142,"(whole milk, bottled water)",(rolls/buns),0.112365,0.349666,0.045151,0.401826,1.149171,0.005861,1.087199,2
4263,"(whole milk, bottled water)",(soda),0.112365,0.313494,0.040021,0.356164,1.136112,0.004795,1.066275,2
4328,"(whole milk, bottled water)",(yogurt),0.112365,0.282966,0.040277,0.358447,1.266753,0.008482,1.117655,2


### Here is an example of what customers purchased in the past if they currently purchased beef.

In [63]:
rules[rules['consequents'] == {"beef"}]

yy = rules[rules['consequents'] == {'beef'}]

#xx = xx[xx['confidence'] == xx['confidence'].max()]

yy = yy[yy['confidence'] >= 0.18]

yy


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2404,"(whole milk, citrus fruit)",(beef),0.092355,0.119548,0.016932,0.183333,1.533548,0.005891,1.078104,2
2422,"(whole milk, fruit/vegetable juice)",(beef),0.062340,0.119548,0.011801,0.189300,1.583461,0.004348,1.086039,2
2482,"(other vegetables, whipped/sour cream)",(beef),0.066957,0.119548,0.012314,0.183908,1.538355,0.004309,1.078863,2
2518,"(soda, pip fruit)",(beef),0.060544,0.119548,0.012314,0.203390,1.701317,0.005076,1.105248,2
2626,"(whole milk, whipped/sour cream)",(beef),0.079785,0.119548,0.014623,0.183280,1.533100,0.005085,1.078033,2
11023,"(soda, rolls/buns, other vegetables)",(beef),0.052591,0.119548,0.010262,0.195122,1.632157,0.003974,1.093894,3
11051,"(whole milk, soda, other vegetables)",(beef),0.069266,0.119548,0.012827,0.185185,1.549038,0.004546,1.080554,3


In [65]:
zz = rules[rules['consequents'] == {'bottled water','whole milk'}]

zz = zz[zz['confidence']>0.2]

In [66]:
zz

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
11350,"(brown bread, other vegetables)","(whole milk, bottled water)",0.059774,0.112365,0.012057,0.201717,1.795187,0.005341,1.111929,2
11476,"(curd, other vegetables)","(whole milk, bottled water)",0.051565,0.112365,0.011544,0.223881,1.992435,0.005750,1.143683,2
11490,"(frankfurter, other vegetables)","(whole milk, bottled water)",0.061057,0.112365,0.012571,0.205882,1.832259,0.005710,1.117762,2
15374,"(rolls/buns, yogurt, other vegetables)","(whole milk, bottled water)",0.052335,0.112365,0.010518,0.200980,1.788634,0.004638,1.110905,3
